In [35]:
import re

dicts: dict[str, dict[int, tuple[int, int]]] = {
    "seed-to-soil": {},
    "soil-to-fertilizer": {},
    "fertilizer-to-water": {},
    "water-to-light": {},
    "light-to-temperature": {},
    "temperature-to-humidity": {},
    "humidity-to-location": {},
}

seed_ids: list[int] = []

# parse file
with open("./input.txt") as input:
    target_dict = dicts["seed-to-soil"]
    while line := input.readline():
        if line.startswith("seeds:"):
            seed_ids = [int(m) for m in re.findall(r"(\d+)", line)]
        elif match := re.match(r"^([^\d\s])+", line):
            target_dict = dicts[match.group()]
        else:
            if match := re.findall(r"(\d+)", line):
                dest_start, src_start, rn = [int(m) for m in match]
                target_dict[src_start] = (dest_start, rn)

# sort dict keys
src_starts = [*dicts.keys()]
for dict_name, next_layer in dicts.items():
    for src_start in sorted(next_layer.keys()):
        next_layer[src_start] = next_layer.pop(src_start)

location_numbers: dict[int, int] = {}

for seed_id in seed_ids:
    path: list[str] = [f"Seed {seed_id}"]
    item_id = seed_id
    for dict_name, stage_dict in dicts.items():
        next_item_id = item_id
        for src_start in stage_dict.keys():
            if src_start > item_id:
                break
            (dest_start, rn) = stage_dict[src_start]
            offset = item_id - src_start
            if rn < offset:
                continue
            if offset < 0:
                raise Exception("bad offset or range?", dict_name, item_id, offset, rn)
            next_item_id = dest_start + offset
        item_id = next_item_id
        path.append(f"{dict_name}: From {item_id} to {next_item_id}.")
    location_numbers[seed_id] = item_id
print(f"Lowest location number: {min(location_numbers.values())}")
# 265018614

Lowest location number: 265018614


In [58]:
import re

dicts: dict[str, dict[int, tuple[int, int]]] = {
    "seed-to-soil": {},
    "soil-to-fertilizer": {},
    "fertilizer-to-water": {},
    "water-to-light": {},
    "light-to-temperature": {},
    "temperature-to-humidity": {},
    "humidity-to-location": {},
}

# parse file
seed_ranges: list[tuple[int, int]] = []
with open("./input.txt") as input:
    target_dict = dicts["seed-to-soil"]
    while line := input.readline():
        if line.startswith("seeds:"):
            seed_nums = [int(m) for m in re.findall(r"(\d+)", line)]
            for i in range(0, len(seed_nums), 2):
                seed_ranges.append((seed_nums[i], seed_nums[i + 1]))
        elif match := re.match(r"^([^\d\s])+", line):
            target_dict = dicts[match.group()]
        else:
            if match := re.findall(r"(\d+)", line):
                dest_start, src_start, rn = [int(m) for m in match]
                target_dict[src_start] = (dest_start, rn)

# sort dict keys
src_starts = [*dicts.keys()]
for dict_name, d in dicts.items():
    for src_start in sorted(d.keys()):
        d[src_start] = d.pop(src_start)

location_numbers: dict[int, int] = {}


def next_layer_pos(src_id: int, stage_dict: dict[int, tuple[int, int]]):
    for src_start in stage_dict.keys():
        if src_start > src_id:
            break
        (dest_start, rn) = stage_dict[src_start]
        offset = src_id - src_start
        if rn <= offset:
            continue
        if offset < 0:
            raise Exception("bad offset or range?", src_id, offset, rn)
        next_layer_pos = dest_start + offset
        further_space = rn - offset - 1
        return next_layer_pos, further_space
    return None


def next_layer_ranges(
    seed_start: int, seed_count: int, stage_dict: dict[int, tuple[int, int]]
):
    next_ranges: list[tuple[int, int]] = []
    while seed_count > 0:
        next_pos = next_layer_pos(seed_start, stage_dict)
        if not next_pos:
            break
        next_start, further_space = next_pos
        seed_cost = min(further_space + 1, seed_count)
        next_ranges.append((next_start, seed_cost))
        seed_start += further_space + 1
        seed_count -= seed_cost
    return next_ranges


def resolve_ranges(
    input_ranges: list[tuple[int, int]], layer_path: list[str] = [*dicts.keys()]
) -> list[tuple[int, int]]:
    if len(layer_path) == 0:
        return input_ranges

    next_layer = dicts[layer_path[0]]
    sub_ranges: list[tuple[int, int]] = []
    for start, count in input_ranges:
        sub_ranges += next_layer_ranges(start, count, next_layer)
    resolved_ranges = resolve_ranges(sub_ranges, layer_path[1:])
    return resolved_ranges


location_ranges = resolve_ranges(seed_ranges)
lowest_location_id = min([id for id, length in location_ranges])

print(lowest_location_id)
# 63179500

63179500
